In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Transformers for Sentiment Analysis
This notebook follows this tutorial: https://github.com/bentrevett/pytorch-sentiment-analysis.

# Loading Tokenizer

In [2]:
from transformers import BertTokenizer

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer.save_pretrained('./transformers_cache/bert-base-uncased/')
tokenizer = BertTokenizer.from_pretrained('./transformers_cache/bert-base-uncased/')
print(len(tokenizer.vocab))

30522


In [23]:
# This will tokenize and lower case the data in a way that is consistent with the pre-trained transformer model.
text = "Hello WORLD how ARE yoU?"
tokens = tokenizer.tokenize(text)
print(tokens)

indexes = tokenizer.convert_tokens_to_ids(tokens)
print(indexes)

indexes = tokenizer.encode(text, add_special_tokens=True)
print(indexes)

['hello', 'world', 'how', 'are', 'you', '?']
[7592, 2088, 2129, 2024, 2017, 1029]
[101, 7592, 2088, 2129, 2024, 2017, 1029, 102]


In [4]:
# `cls_token`: The classifier token which is used when doing sequence classification (classification of the whole
# sequence instead of per-token classification). It is the first token of the sequence when built with special tokens.
init_token = tokenizer.cls_token
# `sep_token`: The separator token, which is used when building a sequence from multiple sequences, e.g. two sequences
# for sequence classification or for a text and a question for question answering. It is also used as the last token of 
# a sequence built with special tokens.
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [5]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [6]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
print(max_input_length)

def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

512


# Preparing Data
Now we define our fields. The transformer expects the batch dimension to be first, so we set `batch_first = True`. As we already have the vocabulary for our text, provided by the transformer we set `use_vocab = False` to tell torchtext that we'll be handling the vocabulary side of things. We pass our `tokenize_and_cut` function as the tokenizer. The `preprocessing` argument is a function that takes in the example after it has been tokenized, this is where we will convert the tokens to their indexes. Finally, we define the special tokens - making note that we are defining them to be their index value and not their string value, i.e. `100` instead of `[UNK]` This is because the sequences will already be converted into indexes.

In [7]:
import torchtext
from torchtext.data import Field, LabelField, BucketIterator

# `use_vocab`: Whether to use a Vocab object. If False, the data in this field should already be numerical.
TEXT = Field(batch_first=True, use_vocab=False, 
             tokenize=tokenize_and_cut, preprocessing=tokenizer.convert_tokens_to_ids, 
             init_token=init_token_idx, eos_token=eos_token_idx, pad_token=pad_token_idx, unk_token=unk_token_idx,
             include_lengths=True)
LABEL = LabelField(dtype=torch.float)

train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL, root='data')
train_data, valid_data = train_data.split()

In [8]:
# Note: The text has already been numericalized. 
print(train_data[0].text[:20])
print(tokenizer.convert_ids_to_tokens(train_data[0].text[:20]))
print(train_data[0].label)

[2061, 1045, 9357, 2006, 1996, 3617, 4942, 29234, 2099, 6833, 2028, 2305, 1037, 3232, 1997, 2086, 3283, 1998, 2245, 1045]
['so', 'i', 'flipped', 'on', 'the', 'digital', 'sub', '##scribe', '##r', 'channels', 'one', 'night', 'a', 'couple', 'of', 'years', 'ago', 'and', 'thought', 'i']
pos


In [9]:
LABEL.build_vocab(train_data)

In [28]:
BATCH_SIZE = 4

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, sort_within_batch=True, device=device)

In [29]:
for batch in train_iterator:
    break
text, text_lens = batch.text
print(text)
print(text_lens)

tensor([[  101,  2178, 17151, 11741, 17743,  1010,  2023,  3972,  6961,  2046,
          1996,  2088,  1997,  1996,  4242,  1998,  1996, 11189,  1010,  1998,
          2009,  2515,  2200,  2092,  1012,  2009,  2987,  1005,  1056,  7001,
          2000,  1996,  2502,  2569,  3896,  2058, 15872,  2066,  2137, 17312,
          2015,  1010,  2009,  7679,  2062,  2006,  6832,  4254,  1012,  1037,
          4659,  3722,  5436,  2008,  9423, 21025, 24700,  3240,  1004,  2522,
          1012,  3288,  2000,  2166,  1012,  2009,  4076,  1996,  2466,  1997,
          1037,  3232,  2040,  4965,  2019,  2214,  2160,  2008,  2001, 10743,
          2188,  2000,  1037,  2200,  2214,  2450,  2040,  2196,  2253,  2648,
          1010,  1998,  3005,  3129,  5419,  1999,  8075,  6214,  1037,  2301,
          3283,  1012,  4326,  2477,  4088,  2000,  4148,  1999,  1996,  2160,
          1010,  1998,  2198,  4205,  4269,  2000,  2735,  2046,  1996,  2158,
          2040,  5419,  1010,  2040,  2001,  2941,  

# Build Model

In [30]:
from transformers import BertModel

# bert = BertModel.from_pretrained('bert-base-uncased')
# bert.save_pretrained('./transformers_cache/bert-base-uncased/')
bert = BertModel.from_pretrained('./transformers_cache/bert-base-uncased/')

In [32]:
# Input: `input_ids`
    # `input_ids`: (batch_size, sequence_length)
# Output: (`last_hidden_state`, `pooler_output`)
    # `last_hidden_state`: (batch_size, sequence_length, hidden_size)
    # Sequence of hidden-states at the output of the last layer of the model.
    # `pooler_output` (batch_size, hidden_size)
    # Last layer hidden-state of the first token of the sequence (classification token)
    # further processed by a Linear layer and a Tanh activation function. The Linear
    # layer weights are trained from the next sentence prediction (classification)
    # objective during pre-training. 

bert_outs, bert_pooled_outs = bert(batch.text[0])
print(batch.text[0].size())
print(bert_outs.size())
print(bert_pooled_outs.size())

torch.Size([4, 137])
torch.Size([4, 137, 768])
torch.Size([4, 768])


In [33]:
# The values at padding positions are NOT zeros?
print(batch.text[0])
print(bert_outs)

tensor([[  101,  2178, 17151, 11741, 17743,  1010,  2023,  3972,  6961,  2046,
          1996,  2088,  1997,  1996,  4242,  1998,  1996, 11189,  1010,  1998,
          2009,  2515,  2200,  2092,  1012,  2009,  2987,  1005,  1056,  7001,
          2000,  1996,  2502,  2569,  3896,  2058, 15872,  2066,  2137, 17312,
          2015,  1010,  2009,  7679,  2062,  2006,  6832,  4254,  1012,  1037,
          4659,  3722,  5436,  2008,  9423, 21025, 24700,  3240,  1004,  2522,
          1012,  3288,  2000,  2166,  1012,  2009,  4076,  1996,  2466,  1997,
          1037,  3232,  2040,  4965,  2019,  2214,  2160,  2008,  2001, 10743,
          2188,  2000,  1037,  2200,  2214,  2450,  2040,  2196,  2253,  2648,
          1010,  1998,  3005,  3129,  5419,  1999,  8075,  6214,  1037,  2301,
          3283,  1012,  4326,  2477,  4088,  2000,  4148,  1999,  1996,  2160,
          1010,  1998,  2198,  4205,  4269,  2000,  2735,  2046,  1996,  2158,
          2040,  5419,  1010,  2040,  2001,  2941,  

Instead of using an embedding layer to get embeddings for our text, we'll be using the pre-trained transformer model. These embeddings will then be fed into a GRU to produce a prediction for the sentiment of the input sentence. We get the embedding dimension size (called the `hidden_size`) from the transformer via its config attribute. The rest of the initialization is standard.

Within the forward pass, we wrap the transformer in a `no_grad` to ensure no gradients are calculated over this part of the model. The transformer actually returns the embeddings for the whole sequence as well as a *pooled* output. The [documentation](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel) states that the pooled output is "usually not a good summary of the semantic content of the input, you’re often better with averaging or pooling the sequence of hidden-states for the whole input sequence", hence we will not be using it. The rest of the forward pass is the standard implementation of a recurrent model, where we take the hidden state over the final time-step, and pass it through a linear layer to get our predictions.

In [39]:
class Classifier(nn.Module):
    def __init__(self, bert, hid_dim, out_dim, n_layers, bidirect, dropout):
        super().__init__()
        # Use `bert` to provide word embeddings. 
        self.bert = bert
        emb_dim = bert.config.hidden_size
        
        self.rnn = nn.GRU(emb_dim, hid_dim, num_layers=n_layers, bidirectional=bidirect, batch_first=True, 
                          dropout=(0 if n_layers < 2 else dropout))
        self.fc = nn.Linear((hid_dim*2 if bidirect else hid_dim), out_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lens):
        # text: (step, batch)
        with torch.no_grad():
            embedded, _ = self.bert(text)

        # `<pad>` token: `bert.config.pad_token_id`
        # Pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lens, batch_first=True)
        # hidden: (num_layers*num_directions, batch, hid_dim)
        packed_outs, hidden = self.rnn(packed_embedded)
        # Unpack sequence, NOT used here. 
        # outs, out_lens = nn.utils.rnn.pad_packed_sequence(packed_outs, batch_first=True)
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat([hidden[-2], hidden[-1]], dim=-1))
        else:
            hidden = self.dropout(hidden[-1])
        return self.fc(hidden)

In [41]:
def count_parameters(model):
    """
    Count trainable parameters. 
    """
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

HID_DIM = 256
OUT_DIM = 1
N_LAYERS = 2
BIDIRECT = True
DROPOUT = 0.25

classifier = Classifier(bert, HID_DIM, OUT_DIM, N_LAYERS, BIDIRECT, DROPOUT).to(device)

print(count_parameters(bert))
print(count_parameters(classifier))

109482240
112241409


In [49]:
# Freeze BERT parameters (NOT train them). 
for name, param in classifier.named_parameters():
    if name.startswith('bert'):
        param.requires_grad_(False)

print(count_parameters(classifier))

2759169


# Train Model

In [50]:
optimizer = optim.Adam(classifier.parameters())
# Binary cross entropy with logits. 
# The binary version of cross entropy loss. 
loss_func = nn.BCEWithLogitsLoss().to(device)

In [51]:
def train_epoch(classifier, iterator, optimizer, loss_func):
    classifier.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        # Forward pass
        text, text_lens = batch.text
        preds = classifier(text, text_lens).squeeze(-1)
        # Calculate loss
        loss = loss_func(preds, batch.label)
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()
        # Accumulate loss and acc
        epoch_loss += loss.item()
        epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(classifier, iterator, loss_func):
    classifier.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            text, text_lens = batch.text
            preds = classifier(text, text_lens).squeeze(-1)
            # Calculate loss
            loss = loss_func(preds, batch.label)
            # Accumulate loss and acc
            epoch_loss += loss.item()
            epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [ ]:
import time
N_EPOCHS = 10
best_valid_loss = np.inf

for epoch in range(N_EPOCHS):
    t0 = time.time()
    train_loss, train_acc = train_epoch(classifier, train_iterator, optimizer, loss_func)
    valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
    epoch_secs = time.time() - t0

    epoch_mins, epoch_secs = int(epoch_secs // 60), int(epoch_secs % 60)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(classifier.state_dict(), 'models/tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
classifier.load_state_dict(torch.load('models/tut6-model.pt'))

valid_loss, valid_acc = eval_epoch(classifier, valid_iterator, loss_func)
test_loss, test_acc = eval_epoch(classifier, test_iterator, loss_func)

print(f'Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}%')
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

# User Input

In [ ]:
def predict_senti(classifier, tokenizer, sentence):
    classifier.eval()
    indexed = tokenizer.encode(text, add_special_tokens=True)
    lens = len(indexed)

    # Note: `batch_first=True`
    indexed = torch.tensor(indexed, dtype=torch.long).unsqueeze(0).to(device)
    lens = torch.tensor(lens, dtype=torch.long).unsqueeze(0).to(device)
    pred = torch.sigmoid(classifier(indexed, lens)).round().type(torch.long)
    return LABEL.vocab.itos[pred.item()]

In [ ]:
predict_senti(classifier, "This is a good film.")

In [52]:
predict_senti(classifier, "This film is terrible.")

NameError: name 'predict_senti' is not defined